In [ ]:
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
import operator

In [ ]:
def json_get_length(path):
    with open(path, encoding="utf-8") as f:
        lines = f.readlines()
        return len(lines)

In [ ]:
def json_to_df(path, l):
    with open(path, encoding="utf-8") as f:
        dic = {}
        for i in tqdm(range(l)):
            line = f.readline()
            dic_new = dict(json.loads(line))
            if i == 0:
                dic = dic_new
                for key in dic:
                    dic[key] = [dic[key]]
            else:
                for key in dic:
                    dic[key].append(dic_new[key])
        f.close()
        dic = pd.DataFrame(dic)
        return dic

In [ ]:
# Read review file
l_review=json_get_length('review.json')
review = json_to_df('review.json',l_review)

In [ ]:
# Read tip file
l_tip=json_get_length('tip.json')
tip = json_to_df('tip.json',l_tip)

In [ ]:
unique_business_id = set(list(review['business_id']))
# Find tips only for these business
tip = tip[tip['business_id'].isin(unique_business_id)]

In [ ]:
# Read business file
l_business=json_get_length('business.json')
business = json_to_df('business.json',l_business)

In [ ]:
# Find information only for these business
business = business[business['business_id'].isin(unique_business_id)]
business.index = range(len(business))

In [ ]:
# Read user file
l_user=json_get_length('user.json')
user = json_to_df('user.json',l_user)

In [ ]:
unique_user_id = set(list(review['user_id']))
# Find users only those who write reviews in review files
user = user[user['user_id'].isin(unique_user_id)]

In [ ]:
# Find information only for bars business
bars_business_id = []
for i in tqdm(range(len(business))):
    if business.loc[i, 'categories'] and business.loc[i, 'review_count'] >= 3 and business.loc[i, 'stars'] and business.loc[i, 'is_open'] == 1 and business.loc[i, 'business_id'] != '#NAME?':
        curr = business.loc[i, 'categories'].split(', ')
        if 'Bars' in curr:
            bars_business_id.append(business.loc[i,'business_id'])
bars_business = business[business['business_id'].isin(bars_business_id)]
bars_business.index = range(len(bars_business))
bars_business.to_csv(r'bars_business.csv')
bars_review = review[review['business_id'].isin(bars_business_id)]
bars_review.index = range(len(bars_review))
bars_review.to_csv(r'bars_review.csv')
bars_tip = tip[tip['business_id'].isin(bars_business_id)]
bars_tip.index = range(len(bars_tip))
bars_tip.to_csv(r'bars_tip.csv')